# 1. Install and Import Dependencies

In [ ]:
!pip3 list

In [ ]:
# Installation for linux, check website to get mac installation
!pip3 install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cpu

In [ ]:
!git clone https://github.com/ultralytics/yolov5

In [ ]:
!pip3 install -r yolov5/requirements.txt

In [ ]:
import torch
from matplotlib import pyplot as plt
import numpy as np
import cv2

# 2. Load Model

In [ ]:
model = torch.hub.load('ultralytics/yolov5', 'yolov5s')

In [ ]:
import numpy as np
import cv2
from IPython import display
from PIL import Image
import matplotlib.pyplot as py
%matplotlib inline
def showVideo(VideoIndex=0, scale=0.5):
    try:
        cap = cv2.VideoCapture(VideoIndex)
    except:
        print("Cannot Open Device")
    try:
        ret, frame = cap.read()
        
        while(ret==True):
            # Capture frame-by-frame
            ret, frame = cap.read()
            
            if not ret:
                # Release the Video Device if ret is false
                cap.release()
                # Message to be displayed after releasing the device
                print ("Released Video Resource")
                break
            #frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            results = model(frame)
            half_frame = cv2.resize(np.squeeze(results.render()), (0, 0), fx = scale, fy = scale)
            #this fixed the pipeline issue as the imshow was converting to 
            #png!!!!
            #credit: 
            # https://medium.com/@kostal91/displaying-real-time-webcam-stream-in-ipython-at-relatively-high-framerate-8e67428ac522
            _,ret_array = cv2.imencode('.jpg', half_frame) 
            i = display.Image(data=ret_array)
            display.display(i)
            display.clear_output(wait=True)
            
    except KeyboardInterrupt:
        # Release the Video Device
        cap.release()
        # Message to be displayed after releasing the device
        print("Released Video Resource from KeyboardInterrupt")
    pass
showVideo(1, 0.5)

In [ ]:
model

# 3. Make Detections with Images

In [ ]:
img = 'https://www.gannett-cdn.com/presto/2022/05/26/NAAS/4926cb75-c0b2-4e8c-a635-1a72a42d5421-Austin_Traffic_jwj_0122.jpg'

In [ ]:
results = model(img)
results.print()

In [ ]:
%matplotlib inline 
plt.imshow(np.squeeze(results.render()))
plt.show()

In [ ]:
results.render()

# 4. Real Time Detections

In [ ]:
cap = cv2.VideoCapture(0)
while cap.isOpened():
    ret, frame = cap.read()
    
    # Make detections 
    results = model(frame)
    
    cv2.imshow('YOLO', np.squeeze(results.render()))
    
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

In [ ]:
cap.release()

# 5. Train from scratch

In [ ]:
import uuid   # Unique identifier
import os
import time

In [ ]:
if not os.path.exists('data'):
    os.mkdir('data')
if not os.path.exists('data/labels'):
    os.mkdir('data/labels')
if not os.path.exists('data/images'):
    os.mkdir('data/images')
IMAGES_PATH = os.path.join('data', 'images') #/data/images
labels = ['andre']
number_imgs = 10

In [ ]:
cap = cv2.VideoCapture(0)
# Loop through labels
for label in labels:
    print('Collecting images for {}'.format(label))
    time.sleep(5)
    
    # Loop through image range
    for img_num in range(number_imgs):
        print('Collecting images for {}, image number {}'.format(label, img_num))
        
        # Webcam feed
        ret, frame = cap.read()
        
        # Naming out image path
        imgname = os.path.join(IMAGES_PATH, label+'.'+str(uuid.uuid1())+'.jpg')
        
        # Writes out image to file 
        cv2.imwrite(imgname, frame)
        
        # Render to the screen
        cv2.imshow('Image Collection', frame)
        
        # 2 second delay between captures
        time.sleep(2)
        
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
cap.release()
cv2.destroyAllWindows()

In [ ]:
print(os.path.join(IMAGES_PATH, labels[0]+'.'+str(uuid.uuid1())+'.jpg'))

In [ ]:
for label in labels:
    print('Collecting images for {}'.format(label))
    for img_num in range(number_imgs):
        print('Collecting images for {}, image number {}'.format(label, img_num))
        imgname = os.path.join(IMAGES_PATH, label+'.'+str(uuid.uuid1())+'.jpg')
        print(imgname)   

In [ ]:
!git clone https://github.com/tzutalin/labelImg

In [ ]:
# commands for linux (check labelImg website for instructions for mac)
!pip install pyqt5 lxml --upgrade
!cd labelImg && pyrcc5 -o libs/resources.py resources.qrc

In [ ]:
# First clear /labelimg/data/predefined_classes.txt and keep it blank
# To initiate the labemImg up run in the console -> python labelImg/labelImg.py
# select "open dir" and choose the data/images folder
# selct "change save dir" and choose the data/labels folder
# change saving model to yolo (bellow save button)
# then save the annotation and do the same for the next images

# at the end go to the labels

# get Dataset in https://github.com/ultralytics/yolov5/wiki/Train-Custom-Data

In [ ]:
!python yolov5/train.py --img 320 --batch 16 --epochs 500 --data yolov5/dataset.yml --weights yolov5s.pt --workers 2

# 6. Load Custom Model

In [ ]:
model = torch.hub.load('ultralytics/yolov5', 'custom', path='yolov5/runs/train/exp/weights/last.pt', force_reload=True)

In [ ]:
img = os.path.join('data', 'images', 'andre.6cce3fd2-5159-11ed-9477-0fd44767cf9f.jpg')

In [ ]:
results = model(img)

In [ ]:
results.print()

In [ ]:
%matplotlib inline 
plt.imshow(np.squeeze(results.render()))
plt.show()

In [ ]:
cap = cv2.VideoCapture(0)
while cap.isOpened():
    ret, frame = cap.read()
    results = model(frame)
    image = frame.copy()
    
    for prediction in results.pred[0].numpy():
        if prediction[4] >= 0.1:
            box = [prediction[0], prediction[1], prediction[2], prediction[3]]
            region = image[int(box[1]):int(box[3]), int(box[0]):int(box[2])]
            # blur the region
            blured_img = cv2.blur(region, (20, 20))
            # replace the blured region in the image
            image[int(box[1]):int(box[3]), int(box[0]):int(box[2])] = blured_img
    # Make detections 
    
    
    cv2.imshow('YOLO', np.squeeze(image))
    
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()